Computer Vision Project:CSL7360

Team Members:
1)Akaash Chatterjee - M24CSE002

2)Alok Dutta - M24CSE032

3)Vishwanath Singh - M24CSE030

4)Prathmesh Chintamani Gosavi - M24CSA023


Topic :Comparative Assessment of Resolution Enhancement Models for Satellite Images
======================================
This code implements and compares various image enhancement techniques:
1. Traditional interpolation methods (bicubic, lanczos)
2. Deep learning-based super-resolution
3. Wavelet-based enhancement
4. Wiener filter deblurring
5. Novel hybrid approaches (Wavelet-SR, Wiener-SR)

Each technique is evaluated using multiple metrics on the UC Merced Land Use dataset.

# 1) Downloading and Importing all Necessary Libraries

In [ ]:
!pip install pywavelets streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
import os
import requests
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, BatchNormalization, Input
from tensorflow.keras.models import Model
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pywt
from scipy.signal import wiener
import time
from tqdm import tqdm

#Data Loading and Pre-processing

In [ ]:
# Define constants
DATASET_URL = "http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip"
DATA_DIR = "UC_Merced_Dataset"
DATASET_PATH = "UC_Merced_Dataset/UCMerced_LandUse/Images"
SELECTED_CATEGORIES = ['agricultural', 'baseballdiamond', 'beach', 'buildings', 'forest',
                       'airplane', 'freeway', 'golfcourse',
                       'harbor',  'mobilehomepark']  # Selected for consistent testing
SAMPLE_COUNT = 1000  # Reduced to manage memory usage
LR_SIZE = 128  # Low-resolution size
HR_SIZE = 256  # High-resolution size
BATCH_SIZE = 16  # Batch size for prediction

In [ ]:
# Create a results directory
RESULTS_DIR = "enhancement_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

def download_uc_merced():
    """Download and extract the UC Merced Land Use dataset"""
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        print("Downloading UC Merced dataset...")
        response = requests.get(DATASET_URL, stream=True)
        zip_path = os.path.join(DATA_DIR, "UCMerced_LandUse.zip")

        with open(zip_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)

        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(DATA_DIR)
        os.remove(zip_path)
        print("UC Merced dataset downloaded and extracted successfully!")
    else:
        print("UC Merced dataset already downloaded.")

In [ ]:
def create_paired_dataset(target_hr_size=256, scale_factor=2):
    """Creates paired low-resolution and high-resolution images for super-resolution training"""
    lr_images = []
    hr_images = []
    original_images = []
    image_paths = []

    lr_size = target_hr_size // scale_factor

    for category in SELECTED_CATEGORIES:
        category_path = os.path.join(DATASET_PATH, category)
        if not os.path.exists(category_path):
            print(f"Warning: Category {category} not found. Skipping...")
            continue

        files = os.listdir(category_path)[:SAMPLE_COUNT]

        for file in files:
            img_path = os.path.join(category_path, file)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Warning: Could not read {file}. Skipping...")
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            original_images.append(img.copy())
            image_paths.append(img_path)

            hr_img = cv2.resize(img, (target_hr_size, target_hr_size))
            hr_images.append(hr_img / 255.0)

            lr_img = cv2.resize(img, (lr_size, lr_size))
            lr_img = cv2.GaussianBlur(lr_img, (3, 3), 0.5)
            lr_images.append(lr_img / 255.0)

    return (np.array(lr_images, dtype=np.float32),
            np.array(hr_images, dtype=np.float32),
            original_images,
            image_paths)

#Enhancement Models


In [ ]:
class InterpolationMethods:
    """Traditional interpolation methods for image upscaling"""

    @staticmethod
    def bicubic(image, scale=2):
        """Apply bicubic interpolation to upscale the image"""
        h, w = image.shape[:2]
        return cv2.resize(image, (w*scale, h*scale), interpolation=cv2.INTER_CUBIC)

    @staticmethod
    def lanczos(image, scale=2):
        """Apply Lanczos interpolation to upscale the image"""
        h, w = image.shape[:2]
        return cv2.resize(image, (w*scale, h*scale), interpolation=cv2.INTER_LANCZOS4)

In [ ]:
class SuperResolutionModel:
    """Deep learning-based super-resolution model"""
    @staticmethod
    def build_generator(scale_factor=2):
        """Build a super-resolution generator model based on CNN architecture"""
        input_layer = Input(shape=(None, None, 3))

        x = Conv2D(64, (3, 3), padding="same")(input_layer)
        x = LeakyReLU(alpha=0.2)(x)

        for _ in range(8):
            skip = x
            x = Conv2D(64, (3, 3), padding="same")(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = Conv2D(64, (3, 3), padding="same")(x)
            x = BatchNormalization()(x)
            x = tf.keras.layers.add([x, skip])

        if scale_factor > 1:
            for _ in range(int(np.log2(scale_factor))):
                x = UpSampling2D(size=2)(x)
                x = Conv2D(32, (3, 3), padding="same")(x)
                x = LeakyReLU(alpha=0.2)(x)

        x = Conv2D(3, (3, 3), padding="same", activation="sigmoid")(x)

        model = Model(input_layer, x)
        return model

    @staticmethod
    def train(model, lr_images, hr_images, epochs=10, batch_size=16):
        """Train the super-resolution model"""
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss="mse")

        history = model.fit(
            lr_images, hr_images,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            verbose=1
        )
        return history


In [ ]:
class EnhancementMethods:
    """Image enhancement techniques"""

    @staticmethod
    def wavelet_sharpening(image):
        """Apply wavelet-based sharpening to enhance image details"""
        img_float = image.astype(np.float32)

        if len(img_float.shape) == 3:
            r, g, b = cv2.split(img_float)
            r_coeffs = pywt.dwt2(r, 'haar')
            g_coeffs = pywt.dwt2(g, 'haar')
            b_coeffs = pywt.dwt2(b, 'haar')
            r_sharp = pywt.idwt2((r_coeffs[0], (r_coeffs[1][0]*1.5, r_coeffs[1][1]*1.5, r_coeffs[1][2]*1.5)), 'haar')
            g_sharp = pywt.idwt2((g_coeffs[0], (g_coeffs[1][0]*1.5, g_coeffs[1][1]*1.5, g_coeffs[1][2]*1.5)), 'haar')
            b_sharp = pywt.idwt2((b_coeffs[0], (b_coeffs[1][0]*1.5, b_coeffs[1][1]*1.5, b_coeffs[1][2]*1.5)), 'haar')
            sharpened = cv2.merge([r_sharp, g_sharp, b_sharp])
        else:
            coeffs = pywt.dwt2(img_float, 'haar')
            sharpened = pywt.idwt2((coeffs[0], (coeffs[1][0]*1.5, coeffs[1][1]*1.5, coeffs[1][2]*1.5)), 'haar')

        return np.clip(sharpened, 0, 255).astype(np.uint8)

    @staticmethod
    def wiener_filter(image, kernel_size=(5, 5)):

        def safe_wiener(channel):
            # Ensure float32 for processing
            channel = channel.astype(np.float32)
            # Apply Wiener filter
            filtered = wiener(channel, kernel_size)
            # Replace NaNs and Infs with zeros or safe values
            filtered = np.nan_to_num(filtered, nan=0.0, posinf=255.0, neginf=0.0)
            # Clip and convert back
            return np.clip(filtered, 0, 255).astype(np.uint8)

        if len(image.shape) == 3:
            r, g, b = cv2.split(image)
            r_deblurred = safe_wiener(r)
            g_deblurred = safe_wiener(g)
            b_deblurred = safe_wiener(b)
            deblurred = cv2.merge([r_deblurred, g_deblurred, b_deblurred])
        else:
            deblurred = safe_wiener(image)

        return deblurred

    # Novel Hybrid Methods
    @staticmethod
    def wavelet_sr_hybrid(sr_image):
        """Combine Super-Resolution with Wavelet Sharpening"""
        img_uint8 = (sr_image * 255).astype(np.uint8)
        sharpened = EnhancementMethods.wavelet_sharpening(img_uint8)
        return sharpened.astype(np.float32) / 255.0

    @staticmethod
    def wiener_sr_hybrid(sr_image):
        """Combine Super-Resolution with Wiener Filter"""
        img_uint8 = (sr_image * 255).astype(np.uint8)
        deblurred = EnhancementMethods.wiener_filter(img_uint8)
        return deblurred.astype(np.float32) / 255.0

#Evaluation Metrics:-

In [ ]:
"""#Evaluation Metrics:-"""

#========== EVALUATION METRICS ==========

class EvaluationMetrics:
    """Methods to evaluate image enhancement quality"""

    @staticmethod
    def calculate_psnr(img1, img2):
        """Calculate Peak Signal-to-Noise Ratio"""
        return psnr(img1, img2)

    @staticmethod
    def calculate_ssim(img1, img2):
        """Calculate Structural Similarity Index"""
        return ssim(img1, img2, data_range=1.0, channel_axis=-1)

    @staticmethod
    def calculate_mse(img1, img2):
        """Calculate Mean Squared Error"""
        return np.mean((img1 - img2) ** 2)

    @staticmethod
    def evaluate_method(enhanced_images, reference_images, method_name):
        """Evaluate a method using multiple metrics"""
        ssim_values = []
        psnr_values = []
        mse_values = []

        for i in range(len(enhanced_images)):
            img1 = enhanced_images[i].astype(np.float32) / 255.0 if enhanced_images[i].max() > 1.0 else enhanced_images[i].astype(np.float32)
            img2 = reference_images[i].astype(np.float32) / 255.0 if reference_images[i].max() > 1.0 else reference_images[i].astype(np.float32)

            ssim_values.append(EvaluationMetrics.calculate_ssim(img1, img2))
            psnr_values.append(EvaluationMetrics.calculate_psnr(img1, img2))
            mse_values.append(EvaluationMetrics.calculate_mse(img1, img2))

        return {
            "method": method_name,
            "ssim": np.mean(ssim_values),
            "psnr": np.mean(psnr_values),
            "mse": np.mean(mse_values)
        }

#Visualization Functions

In [ ]:
"""#Visualization Functions"""

def plot_training_history(history):
    """Plot training history of the super-resolution model"""
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'training_history.png'))
    plt.close()

def visualize_enhancement_methods(images, enhanced_images, method_names, image_index=0):
    """Visualize original and enhanced images"""
    n_methods = len(method_names)
    plt.figure(figsize=(16, 8))
    plt.subplot(2, n_methods, 1)
    plt.imshow(images[image_index])
    plt.title("Original")
    plt.axis('off')
    for i, method_name in enumerate(method_names):
        plt.subplot(2, n_methods, i+2)
        plt.imshow(enhanced_images[i][image_index])
        plt.title(method_name)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'enhancement_comparison_{image_index}.png'))
    plt.close()

def plot_metrics_comparison(metrics_results):
    """Plot comparison of metrics for different methods"""
    methods = [result['method'] for result in metrics_results]
    ssim_values = [result['ssim'] for result in metrics_results]
    psnr_values = [result['psnr'] for result in metrics_results]
    mse_values = [result['mse'] for result in metrics_results]

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    bars = plt.bar(methods, ssim_values, color='skyblue')
    plt.title('SSIM Comparison (higher is better)')
    plt.ylabel('SSIM')
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.3f}', ha='center', va='bottom')

    plt.subplot(1, 3, 2)
    bars = plt.bar(methods, psnr_values, color='lightgreen')
    plt.title('PSNR Comparison (higher is better)')
    plt.ylabel('PSNR (dB)')
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom')

    plt.subplot(1, 3, 3)
    bars = plt.bar(methods, mse_values, color='salmon')
    plt.title('MSE Comparison (lower is better)')
    plt.ylabel('MSE')
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.4f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'metrics_comparison.png'))
    plt.close()

def plot_processing_times(method_names, processing_times):
    """Plot processing times for different methods"""
    plt.figure(figsize=(12, 6))
    bars = plt.bar(method_names, processing_times, color='lightcoral')
    plt.title('Processing Time Comparison')
    plt.ylabel('Time (seconds)')
    plt.xlabel('Enhancement Method')
    plt.xticks(rotation=45)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}s', ha='center', va='bottom')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'processing_time_comparison.png'))
    plt.close()

def compare_degradation_levels(hr_images, lr_datasets, downscale_factors, sample_index=0):
    """Visualize and compare different degradation levels on the same image"""
    plt.figure(figsize=(len(downscale_factors) * 4 + 4, 8))
    plt.subplot(2, len(downscale_factors) + 1, 1)
    plt.imshow(hr_images[sample_index])
    plt.title("Original (HR)")
    plt.axis('off')
    for i, factor in enumerate(downscale_factors):
        plt.subplot(2, len(downscale_factors) + 1, i + 2)
        plt.imshow(lr_datasets[factor][sample_index])
        plt.title(f"{factor}x Downsampled")
        plt.axis('off')
    plt.subplot(2, len(downscale_factors) + 1, len(downscale_factors) + 2)
    plt.imshow(np.ones_like(hr_images[sample_index]) * 0.5)
    plt.title("Difference Maps")
    plt.axis('off')
    for i, factor in enumerate(downscale_factors):
        diff = np.abs(hr_images[sample_index] - lr_datasets[factor][sample_index]) * 3
        diff = np.clip(diff, 0, 1)
        plt.subplot(2, len(downscale_factors) + 1, len(downscale_factors) + i + 3)
        plt.imshow(diff)
        plt.title(f"{factor}x Diff Map")
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f'degradation_comparison_{sample_index}.png'))
    plt.close()
    print("\nDegradation Level Metrics:")
    for factor in downscale_factors:
        ssim_val = EvaluationMetrics.calculate_ssim(lr_datasets[factor][sample_index], hr_images[sample_index])
        psnr_val = EvaluationMetrics.calculate_psnr(lr_datasets[factor][sample_index], hr_images[sample_index])
        print(f"{factor}x Downsampling: SSIM={ssim_val:.4f}, PSNR={psnr_val:.2f}dB")

def visualize_sr_results(lr_images, hr_images, enhanced_images, method_names, sample_indices=[0, 1, 2]):
    """Visualize super-resolution results"""
    n_methods = len(method_names)
    for idx in sample_indices:
        if idx >= len(lr_images):
            continue
        plt.figure(figsize=(20, 10))
        lr_display = cv2.resize(lr_images[idx], (hr_images[idx].shape[1], hr_images[idx].shape[0]))
        plt.subplot(2, n_methods + 1, 1)
        plt.imshow(lr_display)
        plt.title("Low Resolution Input")
        plt.axis('off')
        plt.subplot(2, n_methods + 1, n_methods + 2)
        plt.imshow(hr_images[idx])
        plt.title("High Resolution Ground Truth")
        plt.axis('off')
        for i, method_name in enumerate(method_names):
            plt.subplot(2, n_methods + 1, i + 2)
            plt.imshow(enhanced_images[i][idx])
            plt.title(method_name)
            plt.axis('off')
            ssim_val = EvaluationMetrics.calculate_ssim(enhanced_images[i][idx], hr_images[idx])
            psnr_val = EvaluationMetrics.calculate_psnr(enhanced_images[i][idx], hr_images[idx])
            plt.xlabel(f"SSIM: {ssim_val:.3f}, PSNR: {psnr_val:.2f}dB")
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f'sr_comparison_{idx}.png'))
        plt.close()

#Main function

In [ ]:
def main():
    """Main execution function"""
    download_uc_merced()

    # Create a specific directory for models
    MODEL_SAVE_DIR = "sr_model_files"
    os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

    hr_size = 256
    scale_factor = 2

    print(f"Loading dataset with {scale_factor}x super-resolution scale...")
    lr_images, hr_images, original_images, image_paths = create_paired_dataset(
        target_hr_size=hr_size,
        scale_factor=scale_factor
    )

    print(f"Loaded {len(lr_images)} images from {len(SELECTED_CATEGORIES)} categories")
    print(f"LR shape: {lr_images[0].shape}, HR shape: {hr_images[0].shape}")

    method_names = [
        "Bicubic", "Lanczos", "Super-Resolution",
        "Wavelet", "Wiener",
        "Wavelet-SR", "Wiener-SR"
    ]
    enhanced_images = [[] for _ in range(len(method_names))]
    metrics_results = []
    processing_times = []

    # Apply bicubic interpolation
    print("Applying Bicubic interpolation...")
    start_time = time.time()
    for lr_img in tqdm(lr_images, desc="Bicubic Interpolation"):
        enhanced = InterpolationMethods.bicubic(lr_img, scale=scale_factor)
        enhanced_images[0].append(enhanced)
    processing_times.append(time.time() - start_time)

    # Apply Lanczos interpolation
    print("Applying Lanczos interpolation...")
    start_time = time.time()
    for lr_img in tqdm(lr_images, desc="Lanczos Interpolation"):
        enhanced = InterpolationMethods.lanczos(lr_img, scale=scale_factor)
        enhanced_images[1].append(enhanced)
    processing_times.append(time.time() - start_time)

    # Train and apply Super-Resolution model
    print("Training Super-Resolution model...")
    generator = SuperResolutionModel.build_generator(scale_factor=scale_factor)
    history = SuperResolutionModel.train(generator, lr_images, hr_images, epochs=10, batch_size=8)
    plot_training_history(history)

    # Save the trained model with the proper extension
    model_save_path = os.path.join(MODEL_SAVE_DIR, f"sr_model_x{scale_factor}.keras")
    generator.save(model_save_path)
    print(f"Super-Resolution model saved to {model_save_path}")

    print("Applying Super-Resolution model...")
    start_time = time.time()
    for i in tqdm(range(0, len(lr_images), BATCH_SIZE), desc="Super-Resolution Prediction"):
        batch_lr = lr_images[i:i + BATCH_SIZE]
        batch_lr = np.array(batch_lr)
        predictions = generator.predict(batch_lr, batch_size=BATCH_SIZE, verbose=0)
        for sr_img in predictions:
            sr_img = np.clip(sr_img, 0, 1)
            enhanced_images[2].append(sr_img)
    processing_times.append(time.time() - start_time)

    # Upscale LR images for traditional enhancement methods
    upscaled_lr_images = []
    for lr_img in lr_images:
        upscaled = cv2.resize(lr_img, (hr_size, hr_size))
        upscaled_lr_images.append(upscaled)

    # Apply Wavelet Sharpening
    print("Applying Wavelet Sharpening...")
    start_time = time.time()
    for img in tqdm(upscaled_lr_images, desc="Wavelet Sharpening"):
        img_uint8 = (img * 255).astype(np.uint8)
        enhanced = EnhancementMethods.wavelet_sharpening(img_uint8)
        enhanced = enhanced.astype(np.float32) / 255.0
        enhanced_images[3].append(enhanced)
    processing_times.append(time.time() - start_time)

    # Apply Wiener Filter
    print("Applying Wiener Filter...")
    start_time = time.time()
    for img in tqdm(upscaled_lr_images, desc="Wiener Filter"):
        img_uint8 = (img * 255).astype(np.uint8)
        enhanced = EnhancementMethods.wiener_filter(img_uint8)
        enhanced = enhanced.astype(np.float32) / 255.0
        enhanced_images[4].append(enhanced)
    processing_times.append(time.time() - start_time)

    # Apply Novel Hybrid Methods
    print("Applying Wavelet-SR Hybrid...")
    start_time = time.time()
    for sr_img in tqdm(enhanced_images[2], desc="Wavelet-SR Hybrid"):
        enhanced = EnhancementMethods.wavelet_sr_hybrid(sr_img)
        enhanced_images[5].append(enhanced)
    processing_times.append(time.time() - start_time)

    print("Applying Wiener-SR Hybrid...")
    start_time = time.time()
    for sr_img in tqdm(enhanced_images[2], desc="Wiener-SR Hybrid"):
        enhanced = EnhancementMethods.wiener_sr_hybrid(sr_img)
        enhanced_images[6].append(enhanced)
    processing_times.append(time.time() - start_time)

    # Evaluate all methods
    print("Evaluating all methods...")
    for i, method_name in enumerate(method_names):
        results = EvaluationMetrics.evaluate_method(enhanced_images[i], hr_images, method_name)
        metrics_results.append(results)
        print(f"{method_name}: SSIM={results['ssim']:.4f}, PSNR={results['psnr']:.2f}dB, MSE={results['mse']:.6f}")

    # Visualize results
    print("Generating visualizations...")
    visualize_sr_results(lr_images, hr_images, enhanced_images, method_names)

    # Plot metrics comparison
    plot_metrics_comparison(metrics_results)

    # Plot processing times
    plot_processing_times(method_names, processing_times)

    print(f"All results saved to {RESULTS_DIR} directory")

if __name__ == "__main__":
    main()

Extracting dataset...
UC Merced dataset downloaded and extracted successfully!
Loading dataset with 2x super-resolution scale...
Loaded 1000 images from 10 categories
LR shape: (128, 128, 3), HR shape: (256, 256, 3)
Applying Bicubic interpolation...


Bicubic Interpolation: 100%|██████████| 1000/1000 [00:00<00:00, 3096.70it/s]


Applying Lanczos interpolation...


Lanczos Interpolation: 100%|██████████| 1000/1000 [00:00<00:00, 1166.90it/s]


Training Super-Resolution model...


...

Applying Wavelet-SR Hybrid...


Wavelet-SR Hybrid: 100%|██████████| 1000/1000 [00:07<00:00, 138.54it/s]


Applying Wiener-SR Hybrid...


Wiener-SR Hybrid: 100%|██████████| 1000/1000 [00:18<00:00, 53.07it/s]


Evaluating all methods...
Bicubic: SSIM=0.6756, PSNR=26.49dB, MSE=0.069530
Lanczos: SSIM=0.6963, PSNR=27.02dB, MSE=0.064853
Super-Resolution: SSIM=0.8456, PSNR=27.07dB, MSE=0.002539
Wavelet: SSIM=0.8126, PSNR=28.49dB, MSE=0.002618
Wiener: SSIM=0.6949, PSNR=26.43dB, MSE=0.004028
Wavelet-SR: SSIM=0.8221, PSNR=26.39dB, MSE=0.002962
Wiener-SR: SSIM=0.7447, PSNR=26.00dB, MSE=0.003429
Generating visualizations...
All results saved to enhancement_results directory
